In [1]:
def euklidean_forward(a, b):
    """Returns lists of values of a = m * b + r."""
    a, b = max(a, b), min(a, b)
    a_list, m_list, b_list, r_list = [], [], [], []
    
    r = 1
    while r != 0:
        m = a // b
        r = a % b
        
        for i, i_list in zip([a, m, b, r], [a_list, m_list, b_list, r_list]):
            i_list.append(i)

        a, b = b, r

    return a_list, m_list, b_list, r_list

In [59]:
def output_euklidean_forward(a, b):
    ambr_lists = euklidean_forward(a, b)
    length = len(str(ambr_lists[0][0]))
    
    for a, m, b, r in zip(*ambr_lists):
        print("{:{l}} = {:{l}} * {:{l}} + {:{l}}".format(a, m, b, r, l=length))
    
#     print('-' * (4 * length + 9))
#     print('ggT({}, {}) = {}'.format(ambr_lists[0][0], ambr_lists[2][0], b))

In [34]:
def euklidean_backward(a, b):
    length = len(str(a)) + 1
    ambr_reverse_iterator = zip(*map(reversed, euklidean_forward(a, b)))
    
    a, m_b, r, _ = next(ambr_reverse_iterator)
    m_a, m_b = 0, 1
    
    try:
        a, m_b, b, r = next(ambr_reverse_iterator)
        m_b = -m_b
        m_a = 1
    except StopIteration:
        pass
    print('{:{l}} = {:{l}} * {:{l}} + {:{l}} * {:{l}}'.format(r, m_a, a, m_b, b, l=length))
    
    while True:
        try:
            print('-' * (7 * length + 20))
            a, m, b, r = next(ambr_reverse_iterator)
            print('{:{l}} = {:{l}} * {:{l}} + {:{l}} * ({:{l}} - {:{l}} * {:{l}})'.format('', m_a, b, m_b, a, m, b, l=length))
            m_a, m_b = m_b, m_a - m_b * m
            print('{:{l}} = {:{l}} * {:{l}} + {:{l}} * {:{l}}'.format('', m_a, a, m_b, b, l=length))
        except StopIteration:
            break
    
    return m_a, m_b

In [29]:
def ggT(a, b):
    """Returns biggest common divisor of a and b."""
    a, b = max(a, b), min(a, b)
    
    r = 1
    while r != 0:
        m = a // b
        r = a % b
        a, b = b, r
        
    return a

In [156]:
from math import sqrt

import numpy as np

class RSA():

    def __init__(self, n=100):
        self.n = n
        
    def set_primes(self, p, q):
        self.p, self.q = p, q
        
    def set_encryption_exponent(self, e):
        self.e = e
        self.output_of_draw_coprime()
    
    def sieve_of_eratosthenes(self):
        """Returns a list of prime numbers up to n."""
        prime_candidates = list(range(2, self.n+1))  # list of remaining prime candidates
        prime_index = 0

        while True:
            prime = prime_candidates[prime_index]
            if prime > sqrt(self.n):
                break

            prime_index += 1

            delete_counter = 0
            for index, number in enumerate(prime_candidates[prime_index:], start=prime_index):
                if number % prime == 0:  # if number is divided by prime, delete this number
                    prime_candidates.pop(index - delete_counter)
                    delete_counter += 1 

        return prime_candidates

    def draw_two_primes(self):
        """Draws two (different) big prime numbers up to n."""
        primes = self.sieve_of_eratosthenes()
        big_primes = primes[len(primes) // 2:]  # only consider last fourth of primes

        self.p, self.q = map(int, np.random.choice(big_primes, size=2, replace=False))

    def generate_module(self):
        """Generates a RSA key from two prime numbers up to n."""
        self.N = self.p * self.q
        self.output_of_generate_module()

    def output_of_generate_module(self):
        """Generate text output of the function generate_module."""
        print('First step: Generate module N.')
        print('Draw two prime numbers (p, q) up to {}.'.format(self.n))
        print('p = ', self.p)
        print('q = ', self.q)
        print('N = p * q = ', self.N, '\n')

    def phi(self):
        """Returns phi(N) using p and q, where phi is the euler function."""
        self.phi = (self.p-1) * (self.q-1)
        self.output_of_phi()

    def output_of_phi(self):
        """Generate text output of the function phi."""
        print('Second step: Calculate Phi(N).')
        print('Phi(N) = (p - 1) * (q - 1) = ', self.phi, '\n')
        
    def draw_coprime(self):
        """Draws a coprime number e to Phi(N), which is smaller than Phi(N)."""
        number = self.phi
        while ggT(number, self.phi) > 1:
            number = np.random.randint(2, self.phi)
            
        self.e = number
        self.output_of_draw_coprime()
        
    def output_of_draw_coprime(self):
        """Generate text output of the function draw_coprime."""
        print('Third step: Draw a coprime e (encryption exponent) to Phi(N) with 2 < e < Phi(N).')
        print('e = ', self.e, '\n')
        
    def invert_e(self):
        """Inverts the encryption exponent e using euklidean algorithm.
        Find a number d, with e * d mod Phi(N) = 1."""
        self.output_of_invert_e()

    def output_of_invert_e(self):
        """Generate text output of the function invert_e."""
        print('Fourth step: Calculate the inverse of e in the group Z_{Phi(N)}.')
        print('Find a number d that satisfies the equation e * d mod Phi(N) = 1.')
        print('Euklidean algorithm leads: \n')
        output_euklidean_forward(self.e, self.phi)
        print('\nEuklidean algorithm backwards leads: \n')
        self.d = euklidean_backward(self.e, self.phi)[1] % self.phi
        print('We found:\n1 = u * Phi(N) + d * e'.format())
        print('\nTherefore we have 1 = d * e mod Phi(N), where d = {}.'.format(self.d))
        print('\nPrivate (decryption) key: (d, N) = ({}, {})'.format(self.d, self.N))
        print(' Public (encryption) key: (e, N) = ({}, {})'.format(self.e, self.N))
        
    def encrypt(self, m):
        """Encrypt Messaage m using public key (e, N)."""
        c = pow(m, self.e, self.N)
        self.output_of_encrypt(m, c)
        
        return c
    
    def output_of_encrypt(self, m, c):
        """Generate text output of the function encrypt."""
        print('\n')
        if m >= min(self.p, self.q):
            print('Message {} has to be smaller than p = {} and q = {}!!!'.format(m, self.p, self.q))
        print('Encrypting m = {}:'.format(m))
        print('c = m ^ e mod Phi(N)')
        print('  = {} ^ {} mod {}'.format(m, self.e, self.N))
        print('  = {}'.format(c))
    
    def decrypt(self, c):
        """Decrypt cipher c using private key (d, N)."""
        m = pow(c, self.d, self.N)
        self.output_of_decrypt(c, m)
        
        return m
    
    def output_of_decrypt(self, c, m):
        """Generate text output of the function decrypt."""
        print('\nDecrypting c = {}:'.format(c))
        print('m = c ^ d mod Phi(N)')
        print('  = {} ^ {} mod {}'.format(c, self.d, self.N))
        print('  = {}'.format(m))

In [158]:
rsa = RSA()

rsa.sieve_of_eratosthenes()
rsa.draw_two_primes()
rsa.generate_module()

rsa.phi()

rsa.draw_coprime()

rsa.invert_e()

m = 43
c = rsa.encrypt(m)
rsa.decrypt(c)

First step: Generate module N.
Draw two prime numbers (p, q) up to 100.
p =  53
q =  59
N = p * q =  3127 

Second step: Calculate Phi(N).
Phi(N) = (p - 1) * (q - 1) =  3016 

Third step: Draw a coprime e (encryption exponent) to Phi(N) with 2 < e < Phi(N).
e =  1927 

Fourth step: Calculate the inverse of e in the group Z_{Phi(N)}.
Find a number d that satisfies the equation e * d mod Phi(N) = 1.
Euklidean algorithm leads: 

3016 =    1 * 1927 + 1089
1927 =    1 * 1089 +  838
1089 =    1 *  838 +  251
 838 =    3 *  251 +   85
 251 =    2 *   85 +   81
  85 =    1 *   81 +    4
  81 =   20 *    4 +    1
   4 =    4 *    1 +    0

Euklidean algorithm backwards leads: 

    1 =     1 *    81 +   -20 *     4
-------------------------------------------------------
      =     1 *    81 +   -20 * (   85 -     1 *    81)
      =   -20 *    85 +    21 *    81
-------------------------------------------------------
      =   -20 *    85 +    21 * (  251 -     2 *    85)
      =    21 *   251 

43

In [163]:
rsa = RSA()

rsa.set_primes(7, 11)
rsa.generate_module()

rsa.phi()

rsa.set_encryption_exponent(47)

rsa.invert_e()

m = 2
c = rsa.encrypt(m)
rsa.decrypt(c)

First step: Generate module N.
Draw two prime numbers (p, q) up to 100.
p =  7
q =  11
N = p * q =  77 

Second step: Calculate Phi(N).
Phi(N) = (p - 1) * (q - 1) =  60 

Third step: Draw a coprime e (encryption exponent) to Phi(N) with 2 < e < Phi(N).
e =  47 

Fourth step: Calculate the inverse of e in the group Z_{Phi(N)}.
Find a number d that satisfies the equation e * d mod Phi(N) = 1.
Euklidean algorithm leads: 

60 =  1 * 47 + 13
47 =  3 * 13 +  8
13 =  1 *  8 +  5
 8 =  1 *  5 +  3
 5 =  1 *  3 +  2
 3 =  1 *  2 +  1
 2 =  2 *  1 +  0

Euklidean algorithm backwards leads: 

  1 =   1 *   3 +  -1 *   2
-----------------------------------------
    =   1 *   3 +  -1 * (  5 -   1 *   3)
    =  -1 *   5 +   2 *   3
-----------------------------------------
    =  -1 *   5 +   2 * (  8 -   1 *   5)
    =   2 *   8 +  -3 *   5
-----------------------------------------
    =   2 *   8 +  -3 * ( 13 -   1 *   8)
    =  -3 *  13 +   5 *   8
-----------------------------------------
    =

2